In [1]:
from bs4 import BeautifulSoup as bs
import re
from  selenium import webdriver
import pandas as pd
from urllib.request import urlopen,Request
import time

In [3]:
# Create connection to MongoDB
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['practisedb']
collection = db['cricket']

In [2]:
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

headers={'User-Agent':user_agent,} 

In [5]:
pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',500)

In [7]:
#Getting the abreviation for all the playing countries.For eg, ind for India 
con_abv=pd.read_csv('country_abv.csv',header=None)

In [6]:
#country_abv.csv
# Australia,aus
# Lanka,sl
# India,ind
# Bangladesh,ban
# Pakistan,pak
# Zealand,nz
# Africa,rsa
# England,eng
# Afghanistan,afg
# Indies,wi

In [8]:
#Getting the list of all the matches.
html_sch="https://www.cricbuzz.com/cricket-series/2697/icc-cricket-world-cup-2019/matches"
req=Request(html_sch,headers={'User-Agent': 'Mozilla/5.0'})
url_sch=urlopen(req)
soup_sch=bs(url_sch,'lxml')
sch_a=soup_sch.findAll('a',attrs={'href': re.compile("^/cricket-scores/")})
links=[]
for i in sch_a:
    links.append(i.get('href'))
    
    

In [9]:
#Funtion to find which team win the toss and who elected to bat first
def decide_innings(text,teams):
    cont=[]
    toss=re.search('\w*\s*have won the toss and have opted to\s*\w*',soup.text)
    t=re.split(' ' ,toss.group())
    if t[-1]=='field':
        val=1
    else:
        val=0
    #print(t[0])    
    for con in con_abv[0]:
        #print(con)
        if t[0]==con:
            #print(abv)
            abv=con_abv[con_abv[0]==con][1]
            
           
    for team in teams:
        
        try:
            if abv.iloc[0]==team:
                cont.append([val,team])
            else:
                cont.append([1-val,team])
        except UnboundLocalError:
            cont.append([1-val,team])
            
    return cont        
    

In [ ]:
#getting exact link of the page to Scrap data from 
all_link=[]
[all_link.append(x) for x in links if x not in all_link]
link_final=[]
for i in all_link:
    link_final.append('https://www.cricbuzz.com/cricket-full-commentary' + i[15:])


In [11]:
complete_action=pd.DataFrame(columns=['Over','ball_country','Bowler','bat_country','Batsman','Runs','Ball','Fielding','ball_pos','Velocity'])
counter=0
for txt in link_final:
    driver = webdriver.Firefox()
    driver.get(txt)
    time.sleep(5)
    driver.find_element_by_link_text('Preview').click()
    http_page=driver.page_source
    soup = bs(http_page,'lxml')
    teams=re.search('\w*-vs-\w*',txt)
    teams=re.split('-vs-',teams.group())
    try:
        l=decide_innings(soup.text,teams)
        #print(l)
        for a,bt in sorted(l):
            but_chr=bt.upper()+' Inns'
            bowl_cont=bt.upper()
            bat_cont=[team for team in teams if team not in bt][0].upper()
            driver.find_element_by_link_text(but_chr).click()
            http_page=driver.page_source
            soup = bs(http_page,'lxml')
            all_ball=soup.findAll('div',{"cb-mat-mnu-wrp cb-ovr-num ng-binding"})
            all_action=soup.findAll('p',{"cb-com-ln ng-binding ng-scope cb-col cb-col-90"})
            http_page=driver.page_source
            soup = bs(http_page,'lxml')

            list_of_attributes=[]
            list_of_values=[]
            dict_vals={}
            for a,b in zip(all_ball,all_action):
                list_of_attributes=[]
                list_of_values=[]
                b=re.sub(r"leg byes,"," ",b.text)
                b=re.sub(r"-"," ",b)
                b=re.sub(r" the","",b)
                b=re.sub(r"down","",b)
                b=re.sub(r"'s"," ",b)
                b=re.sub(r"FOUR",'4',b)
                b=re.sub(r"SIX",'6',b)
                b=re.sub(r"3 runs",'3',b)
                b=re.sub(r"2 runs",'2',b)
                b=re.sub(r"1 run",'1',b)
                b=re.sub(r"no run",'0',b)
            
                x=b.split(',',2)
                p=re.split(' to ',x[0])

                counter=counter+1
                bowler=p[0]
                list_of_attributes.append('Over')
                list_of_values.append(a.text)
                list_of_attributes.append('Over_val')
                list_of_values.append(a.text.split('.')[0])
                list_of_attributes.append('bowl_country')
                list_of_values.append(bowl_cont)
                list_of_attributes.append('bowler')
                list_of_values.append(bowler)
                list_of_attributes.append('bat_country')
                list_of_values.append(bat_cont)
                batsman=p[1]
                list_of_attributes.append('batsman')
                list_of_values.append(batsman)
                if x[1].strip().split(' ')[0]!='out':
                    Run=x[1]
                    list_of_attributes.append('Run')
                    list_of_values.append(Run)
                else:
                    Run=x[1].strip().split(' ')[0]
                    list_of_attributes.append('Run')
                    list_of_values.append(Run)
                    Reason=x[1].strip().split(' ')[1]
                    list_of_attributes.append('Reason')
                    list_of_values.append(Reason)


                try:
                    pat=r'(tossed up|flighted|spinning) delivery|short (in length|of length|ball)|fuller|full toss|leg break|flicker|slider|flipper|off break|doosra|carrom|(arm|flighted) ball|teesra|bouncer|inswinger|googly|leg cutter|off cutter|outswinger|yorker|knuckle|slower|[Ss]*hort|ful\[l]* length|good length'
                    if re.search(pat,x[2]):
                        ball_check=re.search(pat,x[2])
                        ball_action=ball_check.group(0)
                        list_of_attributes.append('ball_action')
                        list_of_values.append(ball_action)
                    else:
                        ball_action='Null'
                except IndexError:   
                    ball_action='Null'
                try:
                    pat=r'to the bowler|straight back|straight down|keeper|behind square|slip|slips|fly slip|long stop|long on|third man|gully|deep gully|silly point|point|deep point|covers|cover drive|cover point|extra cover|deep extra cover|silly mid off|mid off|long off|straight over\s*bowler|silly mid on|mid on|long on|forward short leg|short mid wicket|mid\s*wicket|deep mid wicket|sweeper|short square leg|square leg|deep square leg|leg gully|long leg|leg slip|fine\s*leg|short fine|deep fine leg|bowler'
                    if re.search(pat,x[2]):
                        field_check=re.search(pat,x[2])
                        field=field_check.group()
                        list_of_attributes.append('field')
                        list_of_values.append(field)
                    else:
                        field='Null'
                except IndexError:   
                    field='Null'          
                try:
                    pat=r'leg side delivery|away going|pitches it up|on middle|down\s*(the)*\s*leg side|wide of off|[targetting]* on the stump|in towards the stump|outside off|on off stump|down leg|middle and leg|outside off|around the pad|around off|on leg stump|off cutter|flatter|blockhole'
                    if re.search(pat,x[2]):
                        ball_pos=re.search(pat,x[2])
                        ball_pos=ball_pos.group()
                        list_of_attributes.append('ball_pos')
                        list_of_values.append(ball_pos)
                    else:
                        ball_pos='Null'  
                except IndexError:   
                    field='Null'          


                try:
                    pat=r'\d\d\dkph'
                    if re.search(pat,x[2]):
                        Velocity=re.search(pat,x[2])
                        Velocity=Velocity.group()
                        list_of_attributes.append('Velocity')
                        list_of_values.append(Velocity)
                    else:
                        Velocity='Null'
                except IndexError:   
                    Velocity='Null'         

                complete_action.loc[counter]=[a.text,bowl_cont,bowler,bat_cont,batsman,Run,ball_action,field,ball_pos,Velocity]
                dict_vals={key:value for key,value in zip(list_of_attributes,list_of_values) }
                #print(dict_vals)
                collection.insert_one(dict_vals)


    except:
        print('Match Cancelled')
    driver.quit()


Match Cancelled
Match Cancelled
Match Cancelled
Match Cancelled
